# Prepare and understand data for modeling

### Duplicates

Consider the following example.

In [3]:
df = spark.createDataFrame([
        (1, 144.5, 5.9, 33, 'M'),
        (2, 167.2, 5.4, 45, 'M'),
        (3, 124.1, 5.2, 23, 'F'),
        (4, 144.5, 5.9, 33, 'M'),
        (5, 133.2, 5.7, 54, 'F'),
        (3, 124.1, 5.2, 23, 'F'),
        (5, 129.2, 5.3, 42, 'M'),
    ], ['id', 'weight', 'height', 'age', 'gender'])

Check for duplicates.

In [5]:
print('Count of rows: {0}'.format(df.count()))
print('Count of distinct rows: {0}'.format(df.distinct().count()))

Count of rows: 7
Count of distinct rows: 6

If these two numbers differ - you have rows that are exact copies of each other. We can drop these rows by using the `.dropDuplicates(...)` method.

In [7]:
df = df.drop_duplicates()
df.show()

+---+------+------+---+------+
 id|weight|height|age|gender|
+---+------+------+---+------+
 5| 133.2| 5.7| 54| F|
 5| 129.2| 5.3| 42| M|
 1| 144.5| 5.9| 33| M|
 4| 144.5| 5.9| 33| M|
 2| 167.2| 5.4| 45| M|
 3| 124.1| 5.2| 23| F|
+---+------+------+---+------+

Let's confirm.

In [9]:
print('Count of ids: {0}'.format(df.count()))

Count of ids: 6

In [10]:
# 使用与id无关的列进行对比
print('Count of distinct ids: {0}'.format(df.select([c for c in df.columns if c != 'id']).distinct().count()))

Count of distinct ids: 5

We still have one more duplicate. We will use the `.dropDuplicates(...)` but add the `subset` parameter.

In [12]:
# 指定只处理 'id'以外的的列  保留第一个
df = df.dropDuplicates(subset=[c for c in df.columns if c != 'id'])
df.show()

+---+------+------+---+------+
 id|weight|height|age|gender|
+---+------+------+---+------+
 5| 133.2| 5.7| 54| F|
 1| 144.5| 5.9| 33| M|
 2| 167.2| 5.4| 45| M|
 3| 124.1| 5.2| 23| F|
 5| 129.2| 5.3| 42| M|
+---+------+------+---+------+

To calculate the total and distinct number of IDs in one step we can use the `.agg(...)` method.

In [14]:
import pyspark.sql.functions as fn
df.agg(  # 对整个df使用聚合函数, 但不用分组
  fn.count('id').alias('count'),  # alias 取别名
  fn.countDistinct('id').alias('distinct')
).show()

+-----+--------+
count|distinct|
+-----+--------+
 5| 4|
+-----+--------+

Give each row a unique ID.

In [16]:
# Returns a new DataFrame by adding a column or replacing the existing column that has the same name.
df.withColumn('new_id', 
              fn.monotonically_increasing_id()  # 返回一个单调递增的id
             ).show()  

+---+------+------+---+------+-------------+
 id|weight|height|age|gender| new_id|
+---+------+------+---+------+-------------+
 5| 133.2| 5.7| 54| F| 25769803776|
 1| 144.5| 5.9| 33| M| 171798691840|
 2| 167.2| 5.4| 45| M| 592705486848|
 3| 124.1| 5.2| 23| F|1236950581248|
 5| 129.2| 5.3| 42| M|1365799600128|
+---+------+------+---+------+-------------+

### Missing observations

Consider a similar example to the one we presented above.

In [18]:
df_miss = spark.createDataFrame([
        (1, 143.5, 5.6, 28,   'M',  100000),
        (2, 167.2, 5.4, 45,   'M',  None),
        (3, None , 5.2, None, None, None),
        (4, 144.5, 5.9, 33,   'M',  None),
        (5, 133.2, 5.7, 54,   'F',  None),
        (6, 124.1, 5.2, None, 'F',  None),
        (7, 129.2, 5.3, 42,   'M',  76000),
    ], ['id', 'weight', 'height', 'age', 'gender', 'income'])

To find the number of missing observations per row we can use the following snippet.

In [20]:
from pyspark.sql import Row
r = Row(name='aaa', age='22')
r

Out[29]: Row(age='22', name='aaa')

In [21]:
for c in r:
  print(c)

22
aaa

In [22]:
# 寻找每行缺失的数据数量
df_miss.rdd.map(
  lambda row: (row['id'], sum([c == None for c in row]))
).collect()

Out[32]: [(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

Let's see what values are missing so when we count missing observations in columns we can decide whether to drop the observation altogether or impute some of the observations.

In [24]:
df_miss.where("id = 3").show()  # where 是filter的别名

+---+------+------+----+------+------+
 id|weight|height| age|gender|income|
+---+------+------+----+------+------+
 3| null| 5.2|null| null| null|
+---+------+------+----+------+------+

In [25]:
df_miss.where("id == 3").show()

+---+------+------+----+------+------+
 id|weight|height| age|gender|income|
+---+------+------+----+------+------+
 3| null| 5.2|null| null| null|
+---+------+------+----+------+------+

In [26]:
df_miss.filter(df_miss.id == '3').show()

+---+------+------+----+------+------+
 id|weight|height| age|gender|income|
+---+------+------+----+------+------+
 3| null| 5.2|null| null| null|
+---+------+------+----+------+------+

What is the percentage of missing observations we see in each column?

In [28]:
# 检查每列缺失数据的百分比
# agg(*exprs)
df_miss.agg(*[
  (1- (fn.count(c) / fn.count('*'))).alias(c + '_missing')  # count('*')计算group的所有列, count计算非空
  for c in df_miss.columns
]).show()

+----------+------------------+--------------+------------------+------------------+------------------+
id_missing| weight_missing|height_missing| age_missing| gender_missing| income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
 0.0|0.1428571428571429| 0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+

We will drop the `'income'` feature as most of its values are missing.

In [30]:
# 移除income列
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c != 'income'])
df_miss_no_income.show()

+---+------+------+----+------+
 id|weight|height| age|gender|
+---+------+------+----+------+
 1| 143.5| 5.6| 28| M|
 2| 167.2| 5.4| 45| M|
 3| null| 5.2|null| null|
 4| 144.5| 5.9| 33| M|
 5| 133.2| 5.7| 54| F|
 6| 124.1| 5.2|null| F|
 7| 129.2| 5.3| 42| M|
+---+------+------+----+------+

In [31]:
# 使用drop
df_miss.drop('income').show()

+---+------+------+----+------+
 id|weight|height| age|gender|
+---+------+------+----+------+
 1| 143.5| 5.6| 28| M|
 2| 167.2| 5.4| 45| M|
 3| null| 5.2|null| null|
 4| 144.5| 5.9| 33| M|
 5| 133.2| 5.7| 54| F|
 6| 124.1| 5.2|null| F|
 7| 129.2| 5.3| 42| M|
+---+------+------+----+------+

To drop the observations instead you can use the `.dropna(...)` method.

In [33]:
# 移除含3个或以上null的数据项
df_miss_no_income.dropna(thresh=3).show()

+---+------+------+----+------+
 id|weight|height| age|gender|
+---+------+------+----+------+
 1| 143.5| 5.6| 28| M|
 2| 167.2| 5.4| 45| M|
 4| 144.5| 5.9| 33| M|
 5| 133.2| 5.7| 54| F|
 6| 124.1| 5.2|null| F|
 7| 129.2| 5.3| 42| M|
+---+------+------+----+------+

To impute a mean, median or other *calculated* value you need to first calculate the value, create a dict with such values, and then pass it to the `.fillna(...)` method.

In [35]:
# 使用mean值填充null
# 先计算均值 转为pandas dataframe
means = df_miss_no_income.agg(
  *[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c!='gender']
).toPandas()
means

,id,weight,height,age
0,4.0,140.283333,5.471429,40.4


In [36]:
means.to_dict()  # dict like {column -> {index -> value}}

Out[50]: {'id': {0: 4.0},
 'weight': {0: 140.28333333333333},
 'height': {0: 5.471428571428571},
 'age': {0: 40.4}}

In [37]:
means.to_dict('records')  # list like [{column -> value}, ... , {column -> value}]

Out[51]: [{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428571,
 'age': 40.4}]

In [38]:
# 转成dict
means = means.to_dict('records')[0]
means

Out[52]: {'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428571,
 'age': 40.4}

In [39]:
means['gender'] = 'missing'
# 传入一个dict 指定替换值
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
 id| weight|height|age| gender|
+---+------------------+------+---+-------+
 1| 143.5| 5.6| 28| M|
 2| 167.2| 5.4| 45| M|
 3|140.28333333333333| 5.2| 40|missing|
 4| 144.5| 5.9| 33| M|
 5| 133.2| 5.7| 54| F|
 6| 124.1| 5.2| 40| F|
 7| 129.2| 5.3| 42| M|
+---+------------------+------+---+-------+

In [40]:
means = df_miss_no_income.agg(
    *[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c != 'gender']
).toPandas().to_dict('records')[0]

means['gender'] = 'missing'

df_miss_no_income.fillna(means).show()

### Outliers 离群点

Consider another simple example.

In [42]:
df_outliers = spark.createDataFrame([
        (1, 143.5, 5.3, 28),
        (2, 154.2, 5.5, 45),
        (3, 342.3, 5.1, 99),
        (4, 144.5, 5.5, 33),
        (5, 133.2, 5.4, 54),
        (6, 124.1, 5.1, 21),
        (7, 129.2, 5.3, 42),
    ], ['id', 'weight', 'height', 'age'])

First, we calculate the lower and upper *cut off* points for each feature.

In [44]:
df_outliers.describe().show()

+-------+-----------------+------------------+------------------+------------------+
summary| id| weight| height| age|
+-------+-----------------+------------------+------------------+------------------+
 count| 7| 7| 7| 7|
 mean| 4.0|167.28571428571428| 5.314285714285714| 46.0|
 stddev|2.160246899469287| 77.8498004401516|0.1676163419695053|25.845695966640168|
 min| 1| 124.1| 5.1| 21|
 max| 7| 342.3| 5.5| 99|
+-------+-----------------+------------------+------------------+------------------+

In [45]:
df_outliers.toPandas().describe()

,id,weight,height,age
count,7.000000,7.000000,7.000000,7.000000
mean,4.000000,167.285714,5.314286,46.000000
std,2.160247,77.849800,0.167616,25.845696
min,1.000000,124.100000,5.100000,21.000000
25%,2.500000,131.200000,5.200000,30.500000
50%,4.000000,143.500000,5.300000,42.000000
75%,5.500000,149.350000,5.450000,49.500000
max,7.000000,342.300000,5.500000,99.000000


In [46]:
cols = ['weight', 'height', 'age']
bounds = {}
for col in cols:
  quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
  IQR = quantiles[-1] - quantiles[0]
  bounds[col] = [quantiles[0] - 1.5*IQR, quantiles[0], quantiles[1],  quantiles[-1] + 1.5*IQR] 
bounds              

Out[74]: {'weight': [91.69999999999999, 129.2, 154.2, 191.7],
 'height': [4.499999999999999, 5.1, 5.5, 6.1000000000000005],
 'age': [-11.0, 28.0, 54.0, 93.0]}

In [47]:
cols = ['weight', 'height', 'age']
bounds = {}

for col in cols:
    quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]
bounds

Out[76]: {'weight': [91.69999999999999, 191.7],
 'height': [4.499999999999999, 6.1000000000000005],
 'age': [-11.0, 93.0]}

The `bounds` dictionary holds the lower and upper bounds for each feature.

Let's now use it to flag our outliers.

In [50]:
# 选择离群点
outliers = df_outliers.select(*['id']+[
  (
    (df_outliers[c] < bounds[c][0]) |
    (df_outliers[c] > bounds[c][1])
  ).alias(c + '_o') for c in cols
])
outliers.show()

+---+--------+--------+-----+
 id|weight_o|height_o|age_o|
+---+--------+--------+-----+
 1| false| false|false|
 2| false| false|false|
 3| true| false| true|
 4| false| false|false|
 5| false| false|false|
 6| false| false|false|
 7| false| false|false|
+---+--------+--------+-----+

We have two outliers in the `weight` feature and two in the `age` feature.

In [52]:
# 提取这些离群点
df_outliers = df_outliers.join(outliers, on='id')
df_outliers.show()

+---+------+------+---+--------+--------+-----+
 id|weight|height|age|weight_o|height_o|age_o|
+---+------+------+---+--------+--------+-----+
 7| 129.2| 5.3| 42| false| false|false|
 6| 124.1| 5.1| 21| false| false|false|
 5| 133.2| 5.4| 54| false| false|false|
 1| 143.5| 5.3| 28| false| false|false|
 3| 342.3| 5.1| 99| true| false| true|
 2| 154.2| 5.5| 45| false| false|false|
 4| 144.5| 5.5| 33| false| false|false|
+---+------+------+---+--------+--------+-----+

In [53]:
df_outliers.filter('weight_o').select('id', 'weight').show()

+---+------+
 id|weight|
+---+------+
 3| 342.3|
+---+------+

In [54]:
df_outliers.filter('age_o').select('id', 'age').show()

+---+---+
 id|age|
+---+---+
 3| 99|
+---+---+

In [55]:
df_outliers = df_outliers.join(outliers, on='id')
df_outliers.filter('weight_o').select('id', 'weight').show()
df_outliers.filter('age_o').select('id', 'age').show()

## Understand your data

### Descriptive statistics

Load our data and convert it to a Spark DataFrame.

In [58]:
import pyspark.sql.types as typ

Next, we read the data in.

In [60]:
# fraud = sc.textFile('ccFraud.csv.gz')
fraud = sc.textFile("/FileStore/tables/ccFraud_csv.gz")
header = fraud.first()
fraud = fraud\
      .filter(lambda row: row != header)\
      .map(lambda row:[int(elem) for elem in row.split(",")])
fraud.take(2)

Out[100]: [[1, 1, 35, 1, 3000, 4, 14, 2, 0], [2, 2, 2, 1, 0, 9, 0, 18, 0]]

In [61]:
header


Out[102]: '"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"'

Following, we create the schema for our `DataFrame`.

In [63]:
# 自定义数据类型创建DataFrame
fields = [typ.StructField(h[1:-1], typ.IntegerType(), True) for h in header.split(",")]

In [64]:
fields

Out[105]: [StructField(custID,IntegerType,true),
 StructField(gender,IntegerType,true),
 StructField(state,IntegerType,true),
 StructField(cardholder,IntegerType,true),
 StructField(balance,IntegerType,true),
 StructField(numTrans,IntegerType,true),
 StructField(numIntlTrans,IntegerType,true),
 StructField(creditLine,IntegerType,true),
 StructField(fraudRisk,IntegerType,true)]

In [65]:
schema = typ.StructType(fields)

In [66]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(), True)
        for h in header.split(',')
    ]
]

schema = typ.StructType(fields)

Finally, we create our `DataFrame`.

In [68]:
fraud_df = spark.createDataFrame(fraud, schema)

Now that the dataframe is ready we can calculate the basic descriptive statistics for our dataset.

In [70]:
fraud_df.printSchema()

root
-- custID: integer (nullable = true)
-- gender: integer (nullable = true)
-- state: integer (nullable = true)
-- cardholder: integer (nullable = true)
-- balance: integer (nullable = true)
-- numTrans: integer (nullable = true)
-- numIntlTrans: integer (nullable = true)
-- creditLine: integer (nullable = true)
-- fraudRisk: integer (nullable = true)

For categorical columns we will count the frequencies of their values using `.groupby(...)` method.

In [72]:
fraud_df.groupBy('gender').count().show()
# 性别不平衡

+------+-------+
gender| count|
+------+-------+
 1|6178231|
 2|3821769|
+------+-------+

For the truly numerical features we can use the `.describe()` method.

In [74]:
numerical = ['balance', 'numTrans', 'numIntlTrans']

In [75]:
desc = fraud_df.describe(numerical)
desc.show()

+-------+-----------------+------------------+-----------------+
summary| balance| numTrans| numIntlTrans|
+-------+-----------------+------------------+-----------------+
 count| 10000000| 10000000| 10000000|
 mean| 4109.9199193| 28.9351871| 4.0471899|
 stddev|3996.847309737077|26.553781024522852|8.602970115863767|
 min| 0| 0| 0|
 max| 41485| 100| 60|
+-------+-----------------+------------------+-----------------+

Here's how you check skewness (we will do it for the `'balance'` feature only).

In [77]:
# 检查偏度  三阶矩
fraud_df.agg({'balance': 'skewness'}).show() 

+------------------+
 skewness(balance)|
+------------------+
1.1818315552995033|
+------------------+

### Correlations

Calculating correlations in PySpark is very easy once your data is in a DataFrame form.

In [80]:
fraud_df.corr('balance', 'numTrans')  # pearson 相关系数  只支持两者之间

Out[114]: 0.00044523140172659576

In order to create a correlations matrix you can use the script below.

In [82]:
n_numerical = len(numerical)  # 3
corr = []
for i in range(n_numerical):
  temp = [None] * i
  for j in range(i, n_numerical):
    temp.append(fraud_df.corr(numerical[i], numerical[j]))
  corr.append(temp)

### Visualization

First, let's load the modules and set them up.

In [85]:
databricks

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1963878750831123> in <module> 
 ----> 1 databricks

 NameError : name 'databricks' is not defined

In [86]:
""%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import bokeh.charts as chrt
from bokeh.io import output_notebook

output_notebook()

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-1195056615353267> in <module> 
 3 plt . style . use ( 'ggplot' ) 
 4 
 ----> 5 import bokeh . charts as chrt
 6 from bokeh . io import output_notebook
 7 

 ModuleNotFoundError : No module named 'bokeh'

### Histograms

Aggreagate the data in workers and return aggregated list of cut-off points and counts in each bin of the histogram to the driver.

In [88]:
hists = fraud_df.select('balance').rdd.flatMap(lambda row: row).histogram(20)

To plot the histogram you can simply call the matplotlib like below.

In [90]:
data = {
    'bins': hists[0][:-1],
    'freq': hists[1]
}

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(1, 1, 1)
ax.bar(data['bins'], data['freq'], width=2000)
ax.set_title('Histogram of \'balance\'')

plt.savefig('B05793_05_22.png', dpi=300)

In a similar manner, a histogram can be create with Bokeh.

In [92]:
b_hist = chrt.Bar(data, values='freq', label='bins', title='Histogram of \'balance\'')
chrt.show(b_hist)

If your data is small enough to fit on the driver (although we would argue it would normally be faster to use the method showed above) you can bring the data and use the `.hist(...)` (from Matplotlib) or `.Histogram(...)` (from Bokeh) methods.

In [94]:
data_driver = {'obs': fraud_df.select('balance').rdd.flatMap(lambda row: row).collect()}

In [95]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(1, 1, 1)

ax.hist(data_driver['obs'], bins=20)
ax.set_title('Histogram of \'balance\' using .hist()')


plt.savefig('B05793_05_24.png', dpi=300)

In [96]:
b_hist_driver = chrt.Histogram(data_driver, values='obs', title='Histogram of \'balance\' using .Histogram()', bins=20)
chrt.show(b_hist_driver)

### Interactions between features

In this example we will sample our fraud dataset at 1% given gender as strata.

In [99]:
data_sample = fraud_df.sampleBy('gender', {1: 0.0002, 2: 0.0002}).select(numerical)

To put multiple 2D charts in one go you can use

In [101]:
data_multi = dict([
    (elem, data_sample.select(elem).rdd.flatMap(lambda row: row).collect()) 
    for elem in numerical
])

sctr = chrt.Scatter(data_multi, x='balance', y='numTrans')

chrt.show(sctr)